In [66]:
import os
import glob
import re
import geopandas as gpd
import numpy as np
import rasterio
from shapely.geometry import mapping
from rasterio.features import geometry_mask
from scipy.interpolate import NearestNDInterpolator
from functions import write_output
import tqdm
import startinpy
import time
from rasterio import Affine

def get_bbox(raster_paths):
    bboxes = []
    for raster_path in raster_paths:
        with rasterio.open(raster_path) as src:
            bbox = src.bounds
            bboxes.append(bbox)
    left = max([bbox.left for bbox in bboxes])
    bottom = max([bbox.bottom for bbox in bboxes])
    right = min([bbox.right for bbox in bboxes])
    top = min([bbox.top for bbox in bboxes])
    return rasterio.coords.BoundingBox(left, bottom, right, top)

def crop_raster(raster_path, bbox, no_data=-9999, file_number=None, tile=None):
    with rasterio.open(raster_path) as src:
        window = src.window(bbox.left, bbox.bottom, bbox.right, bbox.top)

        # Read the data for the specified window
        cropped_data = src.read(window=window)

        # Ensure window attributes are integers (sometimes bbox in float)
        row_off = int(window.row_off)
        col_off = int(window.col_off)
        height = int(window.height)
        width = int(window.width)

        # Replace no-data values in the data
        if src.nodata is not None:
            cropped_data[cropped_data == src.nodata] = no_data

        # Check for specific tile conditions for cropping
        if tile == "25DN2" and file_number in ["25", "24"]:
            cropped_data = cropped_data[:, :2150, :]  

        return cropped_data, src.window_transform(window), src.crs
    
def extract_center_cells(cropped_data, no_data=-9999):
    # Get coordinates of center cells
    cropped_data = cropped_data[0, :, :]

    # Get the indices of the rows and columns
    rows, cols = np.indices(cropped_data.shape)

    # Identify corner coordinates
    corners = {
        "top_left": (0, 0),
        "top_right": (0, cropped_data.shape[1] - 1),
        "bottom_left": (cropped_data.shape[0] - 1, 0),
        "bottom_right": (cropped_data.shape[0] - 1, cropped_data.shape[1] - 1)
    }
    
    # Mask for valid center cells (non-no_data)
    valid_center_cells = (cropped_data != no_data) 

    # Extract x, y, z values for valid cells
    x_valid = cols[valid_center_cells]
    y_valid = rows[valid_center_cells]
    z_valid = cropped_data[valid_center_cells]

    # Create interpolator from valid points
    interpolator = NearestNDInterpolator(list(zip(x_valid, y_valid)), z_valid)

    # Check each corner for no data and interpolate if necessary
    for corner_name, (row, col) in corners.items():
        if cropped_data[row, col] == no_data:
            # Interpolate the nearest valid value
            cropped_data[row, col] = interpolator((col, row))

    # Extract non-no_data and center cells again after filling corners
    valid_center_cells = (cropped_data != no_data)

    # Extract final x, y, z values after filling corners
    x_filled = cols[valid_center_cells]
    y_filled = rows[valid_center_cells]
    z_filled = cropped_data[valid_center_cells]

    # Prepare final list of [x, y, z]
    xyz_filled = []
    for x_i, y_i, z_i in zip(x_filled, y_filled, z_filled):
        xyz_filled.append([x_i, y_i, z_i])

    return xyz_filled


def fill_raster(cropped_data, nodata_value, transform):
    # creating delaunay
    points = extract_center_cells(cropped_data, no_data=nodata_value)
    dt = startinpy.DT()
    dt.insert(points, "BBox")
    print("dt")

    # now interpolation
    new_data = np.copy(cropped_data)

    # for interpolation, grid of all column and row positions, excluding the first and last rows/cols
    cols, rows = np.meshgrid(
        np.arange(1, cropped_data.shape[2] - 1),
        np.arange(1, cropped_data.shape[1] - 1)

    )

    # flatten the grid to get a list of all (col, row) locations
    locs = np.column_stack((cols.ravel(), rows.ravel()))

    # laplace interpolation
    interpolated_values = dt.interpolate({"method": "Laplace"}, locs)

    # reshape interpolated grid back to og
    interpolated_grid = np.reshape(interpolated_values, (cropped_data.shape[1] - 2, cropped_data.shape[2] - 2))

    # fill new_data with interpolated values
    new_data[0, 1:-1, 1:-1] = interpolated_grid
    new_data = np.where(np.isnan(new_data), nodata_value, new_data)
    
    new_transform = transform * Affine.translation(1, 1)

    return new_data[0, 1:-1, 1:-1], new_transform

def chm_finish(chm_array, dtm_array, transform):
    result_array = chm_array[0, 1:-1, 1:-1] - dtm_array
    result_array[(result_array < 2) | (result_array > 40)] = 0
    
    new_transform = transform * Affine.translation(1, 1)

    return result_array, new_transform

def replace_buildings(filled_dtm, dsm_buildings, buildings_geometries, transform, nodata_value=-9999):

    building_mask = geometry_mask(buildings_geometries, transform=transform, invert=False, out_shape=filled_dtm.shape)

    # Apply the mask to the filled DTM
    final_dtm = np.where((building_mask), filled_dtm, dsm_buildings)

    return final_dtm


def load_buildings(buildings_path, layer):
     buildings_gdf = gpd.read_file(buildings_path, layer = layer)
     return [mapping(geom) for geom in buildings_gdf.geometry]

def extract_tilename(filename):
    match = re.match(r'CHM_(\w+)_\d+\.TIF', filename)
    if match:
        return match.group(1)
    return None


def process_files(chm_files, dtm_path, dsm_path, buildings_path, output_base_folder, nodata_value=-9999):
    first_chm_filename = os.path.basename(chm_files[0])
    tile = extract_tilename(first_chm_filename)

    if not tile:
        print(f"Skipping {first_chm_filename}, couldn't extract common part.")
        return

    # building geom loaded in once
    building_geometries = load_buildings(buildings_path, layer=tile)

    # output folder for tile
    output_folder = os.path.join(output_base_folder, tile)
    os.makedirs(output_folder, exist_ok=True)
    total_start_time = time.time()


    # Use tqdm to wrap the file list for progress tracking
    for chm_path in tqdm.tqdm(chm_files, desc="Processing Files", unit="file"):
        chm_filename = os.path.basename(chm_path)
        file_number = re.search(r'_(\d+)\.TIF', chm_filename).group(1)
        print(file_number)

        # overlap BBOX dtm, dsm & chm
        raster_paths = [dtm_path, chm_path, dsm_path]
        overlapping_bbox = get_bbox(raster_paths)
        print(f"\nProcessing tile {chm_filename}")
        start_time = time.time()

        # Cropping rasters to bbox
        dtm_cropped, dtm_transform, dtm_crs = crop_raster(dtm_path, overlapping_bbox, no_data=nodata_value, tile=tile, file_number=file_number)
        dsm_cropped, dsm_transform, dsm_crs = crop_raster(dsm_path, overlapping_bbox, no_data=nodata_value, tile=tile, file_number=file_number)
        chm_cropped, chm_transform, _ = crop_raster(chm_path, overlapping_bbox, no_data=nodata_value,  tile=tile, file_number=file_number)
        
       
        print(f"cropped at {(time.time() - start_time):.2f}")
        # no data values fill

        # fill
        filled_dtm, _ = fill_raster(dtm_cropped, nodata_value, dtm_transform)
        print(f"filled dtm at {(time.time() - start_time):.2f}")

        filled_dsm, new_dsm_transform = fill_raster(dsm_cropped, nodata_value, dsm_transform)
        print(f"filled dsm at {(time.time() - start_time):.2f}")
        # # norm CHM calculation
        chm_result, new_chm_transform = chm_finish(chm_cropped, filled_dtm, chm_transform)
        #
        # Insert buildings from DSM in DTM
        final_dsm_with_buildings = replace_buildings(filled_dtm, filled_dsm, building_geometries, new_dsm_transform,
                                                     nodata_value)
        print(f"filled buildings at {(time.time() - start_time):.2f}")

        # output file names
        output_dtm_filename = f"DSM_{tile}_{file_number}.tif"
        output_chm_filename = f"CHM_{tile}_{file_number}.tif"

        # Saving final DTM + buildings
        output_dtm_path = os.path.join(output_folder, output_dtm_filename)
        write_output(rasterio.open(dtm_path), final_dsm_with_buildings, new_dsm_transform, output_dtm_path, True)

        # FOR NOW SAVING TO NEW PATH FOR TESTING -> CHANGE THIS WHEN IT WORKS
        output_chm_path = os.path.join(output_folder, output_chm_filename)
        write_output(rasterio.open(chm_path), chm_result, new_chm_transform, output_chm_path, True)

        elapsed_time = time.time() - start_time
        print(f"Processed {chm_filename} in {elapsed_time:.2f} seconds and saved output to {output_dtm_filename}")
    
    total_elapsed_time = time.time() - total_start_time
    print(f"\nAll files processed in {total_elapsed_time:.2f} seconds.")


In [69]:
geotiff_dtm = "data/DTM_ams.tif"
geotiff_dsm = "data/DSM_ams.tif"
buildings = "data/ams_buildings.gpkg"


In [68]:
chm_folder = "D:/Geomatics/CHM/25DN2"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

Processing Files:   0%|          | 0/22 [00:00<?, ?file/s]

25

Processing tile CHM_25DN2_25.TIF
cropped at 12.92
dt
filled dtm at 121.26
dt
filled dsm at 292.34
filled buildings at 293.19


Processing Files:   5%|▍         | 1/22 [04:54<1:43:02, 294.43s/file]

File written to 'D:/Geomatics/final\25DN2\DSM_25DN2_25.tif'
File written to 'D:/Geomatics/final\25DN2\CHM_25DN2_25.tif'
Processed CHM_25DN2_25.TIF in 294.27 seconds and saved output to DSM_25DN2_25.tif
24

Processing tile CHM_25DN2_24.TIF
cropped at 13.75
dt
filled dtm at 149.39
dt
filled dsm at 320.56
filled buildings at 321.01


Processing Files:   9%|▉         | 2/22 [10:16<1:43:33, 310.70s/file]

...

File written to 'D:/Geomatics/final\25DN2\DSM_25DN2_03.tif'
File written to 'D:/Geomatics/final\25DN2\CHM_25DN2_03.tif'
Processed CHM_25DN2_03.TIF in 245.78 seconds and saved output to DSM_25DN2_03.tif
02

Processing tile CHM_25DN2_02.TIF
cropped at 8.71
dt
filled dtm at 404.90
dt
filled dsm at 552.54
filled buildings at 552.93


Processing Files:  95%|█████████▌| 21/22 [2:00:09<05:49, 349.12s/file]

File written to 'D:/Geomatics/final\25DN2\DSM_25DN2_02.tif'
File written to 'D:/Geomatics/final\25DN2\CHM_25DN2_02.tif'
Processed CHM_25DN2_02.TIF in 553.99 seconds and saved output to DSM_25DN2_02.tif
01

Processing tile CHM_25DN2_01.TIF
cropped at 8.82
dt
filled dtm at 97.73
dt
filled dsm at 224.73
filled buildings at 225.10


Processing Files: 100%|██████████| 22/22 [2:03:56<00:00, 338.00s/file]

File written to 'D:/Geomatics/final\25DN2\DSM_25DN2_01.tif'
File written to 'D:/Geomatics/final\25DN2\CHM_25DN2_01.tif'
Processed CHM_25DN2_01.TIF in 226.15 seconds and saved output to DSM_25DN2_01.tif

All files processed in 7436.11 seconds.


In [70]:
chm_folder = "D:/Geomatics/CHM/25GN1"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

Processing Files:   0%|          | 0/25 [00:00<?, ?file/s]

01

Processing tile CHM_25GN1_01.TIF
cropped at 8.92
dt
filled dtm at 77.12
dt
filled dsm at 170.38
filled buildings at 170.92


Processing Files:   4%|▍         | 1/25 [02:52<1:08:48, 172.00s/file]

File written to 'D:/Geomatics/final\25GN1\DSM_25GN1_01.tif'
File written to 'D:/Geomatics/final\25GN1\CHM_25GN1_01.tif'
Processed CHM_25GN1_01.TIF in 171.96 seconds and saved output to DSM_25GN1_01.tif
08

Processing tile CHM_25GN1_08.TIF
cropped at 9.32
dt
filled dtm at 78.37
dt
filled dsm at 180.48
filled buildings at 181.01


Processing Files:   8%|▊         | 2/25 [05:54<1:08:12, 177.95s/file]

...

File written to 'D:/Geomatics/final\25GN1\DSM_25GN1_22.tif'
File written to 'D:/Geomatics/final\25GN1\CHM_25GN1_22.tif'
Processed CHM_25GN1_22.TIF in 346.04 seconds and saved output to DSM_25GN1_22.tif
23

Processing tile CHM_25GN1_23.TIF
cropped at 13.52
dt
filled dtm at 135.61
dt
filled dsm at 277.24
filled buildings at 277.91


Processing Files:  96%|█████████▌| 24/25 [1:53:08<05:06, 306.41s/file]

File written to 'D:/Geomatics/final\25GN1\DSM_25GN1_23.tif'
File written to 'D:/Geomatics/final\25GN1\CHM_25GN1_23.tif'
Processed CHM_25GN1_23.TIF in 278.97 seconds and saved output to DSM_25GN1_23.tif
24

Processing tile CHM_25GN1_24.TIF
cropped at 12.99
dt
filled dtm at 125.59
dt
filled dsm at 301.73
filled buildings at 302.38


Processing Files: 100%|██████████| 25/25 [1:58:12<00:00, 283.68s/file]

File written to 'D:/Geomatics/final\25GN1\DSM_25GN1_24.tif'
File written to 'D:/Geomatics/final\25GN1\CHM_25GN1_24.tif'
Processed CHM_25GN1_24.TIF in 303.44 seconds and saved output to DSM_25GN1_24.tif

All files processed in 7092.08 seconds.


In [71]:
chm_folder = "D:/Geomatics/CHM/25BZ1"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

Processing Files:   0%|          | 0/23 [00:00<?, ?file/s]

23

Processing tile CHM_25BZ1_23.TIF
cropped at 13.69
dt
filled dtm at 114.22
dt
filled dsm at 270.45
filled buildings at 270.63


Processing Files:   4%|▍         | 1/23 [04:31<1:39:38, 271.77s/file]

File written to 'D:/Geomatics/final\25BZ1\DSM_25BZ1_23.tif'
File written to 'D:/Geomatics/final\25BZ1\CHM_25BZ1_23.tif'
Processed CHM_25BZ1_23.TIF in 271.71 seconds and saved output to DSM_25BZ1_23.tif
01

Processing tile CHM_25BZ1_01.TIF
cropped at 13.11
dt
filled dtm at 153.41
dt
filled dsm at 357.31
filled buildings at 357.54


Processing Files:   9%|▊         | 2/23 [10:30<1:53:01, 322.91s/file]

...

File written to 'D:/Geomatics/final\25BZ1\DSM_25BZ1_19.tif'
File written to 'D:/Geomatics/final\25BZ1\CHM_25BZ1_19.tif'
Processed CHM_25BZ1_19.TIF in 316.31 seconds and saved output to DSM_25BZ1_19.tif
24

Processing tile CHM_25BZ1_24.TIF
cropped at 8.56
dt
filled dtm at 118.34
dt
filled dsm at 230.00
filled buildings at 230.13


Processing Files:  96%|█████████▌| 22/23 [4:20:42<06:27, 387.49s/file]

File written to 'D:/Geomatics/final\25BZ1\DSM_25BZ1_24.tif'
File written to 'D:/Geomatics/final\25BZ1\CHM_25BZ1_24.tif'
Processed CHM_25BZ1_24.TIF in 231.17 seconds and saved output to DSM_25BZ1_24.tif
25

Processing tile CHM_25BZ1_25.TIF
cropped at 8.61
dt
filled dtm at 110.63
dt
filled dsm at 251.86
filled buildings at 251.98


Processing Files: 100%|██████████| 23/23 [4:24:55<00:00, 691.10s/file]

File written to 'D:/Geomatics/final\25BZ1\DSM_25BZ1_25.tif'
File written to 'D:/Geomatics/final\25BZ1\CHM_25BZ1_25.tif'
Processed CHM_25BZ1_25.TIF in 253.03 seconds and saved output to DSM_25BZ1_25.tif

All files processed in 15895.23 seconds.


In [72]:
chm_folder = "D:/Geomatics/CHM/25BZ2"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

Processing Files:   0%|          | 0/21 [00:00<?, ?file/s]

01

Processing tile CHM_25BZ2_01.TIF
cropped at 8.68
dt
filled dtm at 217.49
dt
filled dsm at 424.59
filled buildings at 424.78


Processing Files:   5%|▍         | 1/21 [07:05<2:21:57, 425.88s/file]

File written to 'D:/Geomatics/final\25BZ2\DSM_25BZ2_01.tif'
File written to 'D:/Geomatics/final\25BZ2\CHM_25BZ2_01.tif'
Processed CHM_25BZ2_01.TIF in 425.83 seconds and saved output to DSM_25BZ2_01.tif
02

Processing tile CHM_25BZ2_02.TIF
cropped at 8.53
dt
filled dtm at 190.81
dt
filled dsm at 358.01
filled buildings at 358.21


Processing Files:  10%|▉         | 2/21 [13:05<2:02:27, 386.72s/file]

...

File written to 'D:/Geomatics/final\25BZ2\DSM_25BZ2_23.tif'
File written to 'D:/Geomatics/final\25BZ2\CHM_25BZ2_23.tif'
Processed CHM_25BZ2_23.TIF in 233.87 seconds and saved output to DSM_25BZ2_23.tif
24

Processing tile CHM_25BZ2_24.TIF
cropped at 8.57
dt
filled dtm at 89.45
dt
filled dsm at 235.22
filled buildings at 235.42


Processing Files:  95%|█████████▌| 20/21 [2:54:03<04:58, 298.96s/file]

File written to 'D:/Geomatics/final\25BZ2\DSM_25BZ2_24.tif'
File written to 'D:/Geomatics/final\25BZ2\CHM_25BZ2_24.tif'
Processed CHM_25BZ2_24.TIF in 236.47 seconds and saved output to DSM_25BZ2_24.tif
13

Processing tile CHM_25BZ2_13.TIF
cropped at 8.56
dt
filled dtm at 476.12
dt
filled dsm at 689.48
filled buildings at 689.67


Processing Files: 100%|██████████| 21/21 [3:05:34<00:00, 530.21s/file]

File written to 'D:/Geomatics/final\25BZ2\DSM_25BZ2_13.tif'
File written to 'D:/Geomatics/final\25BZ2\CHM_25BZ2_13.tif'
Processed CHM_25BZ2_13.TIF in 690.72 seconds and saved output to DSM_25BZ2_13.tif

All files processed in 11134.40 seconds.


In [73]:
chm_folder = "D:/Geomatics/CHM/25DN1"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

Processing Files:   0%|          | 0/13 [00:00<?, ?file/s]

20

Processing tile CHM_25DN1_20.TIF
cropped at 8.59
dt
filled dtm at 84.14
dt
filled dsm at 215.36
filled buildings at 215.57


Processing Files:   8%|▊         | 1/13 [03:36<43:19, 216.66s/file]

File written to 'D:/Geomatics/final\25DN1\DSM_25DN1_20.tif'
File written to 'D:/Geomatics/final\25DN1\CHM_25DN1_20.tif'
Processed CHM_25DN1_20.TIF in 216.62 seconds and saved output to DSM_25DN1_20.tif
02

Processing tile CHM_25DN1_02.TIF
cropped at 9.03
dt
filled dtm at 154.12
dt
filled dsm at 313.79
filled buildings at 313.98


Processing Files:  15%|█▌        | 2/13 [08:51<50:20, 274.56s/file]

...

File written to 'D:/Geomatics/final\25DN1\DSM_25DN1_14.tif'
File written to 'D:/Geomatics/final\25DN1\CHM_25DN1_14.tif'
Processed CHM_25DN1_14.TIF in 190.44 seconds and saved output to DSM_25DN1_14.tif
15

Processing tile CHM_25DN1_15.TIF
cropped at 8.92
dt
filled dtm at 71.61
dt
filled dsm at 179.82
filled buildings at 180.04


Processing Files:  92%|█████████▏| 12/13 [44:06<03:21, 201.49s/file]

File written to 'D:/Geomatics/final\25DN1\DSM_25DN1_15.tif'
File written to 'D:/Geomatics/final\25DN1\CHM_25DN1_15.tif'
Processed CHM_25DN1_15.TIF in 181.10 seconds and saved output to DSM_25DN1_15.tif
16

Processing tile CHM_25DN1_16.TIF
cropped at 8.66
dt
filled dtm at 185.94
dt
filled dsm at 363.09
filled buildings at 363.31


Processing Files: 100%|██████████| 13/13 [50:10<00:00, 231.60s/file]

File written to 'D:/Geomatics/final\25DN1\DSM_25DN1_16.tif'
File written to 'D:/Geomatics/final\25DN1\CHM_25DN1_16.tif'
Processed CHM_25DN1_16.TIF in 364.36 seconds and saved output to DSM_25DN1_16.tif

All files processed in 3010.82 seconds.


In [74]:
chm_folder = "D:/Geomatics/CHM/25DN2"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

Processing Files:   0%|          | 0/22 [00:00<?, ?file/s]

25

Processing tile CHM_25DN2_25.TIF
cropped at 8.69
dt
filled dtm at 80.96
dt
filled dsm at 210.48
filled buildings at 210.83


Processing Files:   5%|▍         | 1/22 [03:32<1:14:29, 212.82s/file]

File written to 'D:/Geomatics/final\25DN2\DSM_25DN2_25.tif'
File written to 'D:/Geomatics/final\25DN2\CHM_25DN2_25.tif'
Processed CHM_25DN2_25.TIF in 211.91 seconds and saved output to DSM_25DN2_25.tif
24

Processing tile CHM_25DN2_24.TIF
cropped at 8.69
dt
filled dtm at 140.07
dt
filled dsm at 305.88
filled buildings at 306.23


Processing Files:   9%|▉         | 2/22 [08:40<1:29:28, 268.42s/file]

...

File written to 'D:/Geomatics/final\25DN2\DSM_25DN2_04.tif'
File written to 'D:/Geomatics/final\25DN2\CHM_25DN2_04.tif'
Processed CHM_25DN2_04.TIF in 251.37 seconds and saved output to DSM_25DN2_04.tif
03

Processing tile CHM_25DN2_03.TIF
cropped at 8.66
dt
filled dtm at 144.55
dt
filled dsm at 288.74
filled buildings at 289.24


Processing Files:  91%|█████████ | 20/22 [1:37:11<09:05, 272.98s/file]

File written to 'D:/Geomatics/final\25DN2\DSM_25DN2_03.tif'
File written to 'D:/Geomatics/final\25DN2\CHM_25DN2_03.tif'
Processed CHM_25DN2_03.TIF in 290.34 seconds and saved output to DSM_25DN2_03.tif
02

Processing tile CHM_25DN2_02.TIF
cropped at 11.10
dt


Processing Files:  91%|█████████ | 20/22 [1:44:18<10:25, 312.93s/file]


KeyboardInterrupt: 

In [ ]:
chm_folder = "D:/Geomatics/CHM/25DN2_last"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
chm_folder = "D:/Geomatics/CHM/25EZ2"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
chm_folder = "D:/Geomatics/CHM/25EZ1"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
chm_folder = "D:/Geomatics/CHM/25FZ1"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
chm_folder = "D:/Geomatics/CHM/25HZ1"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
chm_folder = "D:/Geomatics/CHM/25GN2"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
chm_folder = "D:/Geomatics/CHM/25GZ2"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
chm_folder = "D:/Geomatics/CHM/25GZ1"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
chm_folder = "D:/Geomatics/CHM/25HN1"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
chm_folder = "D:/Geomatics/CHM/25HZ2"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
chm_folder = "D:/Geomatics/CHM/25HZ1"
output_base_folder = "D:/Geomatics/final"

chm_files = glob.glob(os.path.join(chm_folder, "*.TIF"))
process_files(chm_files, geotiff_dtm, geotiff_dsm, buildings, output_base_folder)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def fill_raster2(cropped_data, mask, nodata_value, transform):
    # creating delaunay
    points = extract_center_cells(cropped_data, mask, no_data=nodata_value)
    dt = startinpy.DT()
    dt.insert(points, "BBox")
    print("dt")
    

    
    pts = dt.points
    trs = dt.triangles
    
    # Create a new figure with a specified size (width, height) in inches
    plt.figure(figsize=(20, 25))  # Adjust the size as needed
    
    # Plot the triangles with thinner lines
    plt.triplot(pts[:, 0], pts[:, 1], trs, linewidth=0.1)  # Adjust the linewidth as needed
    
    # Plot the points, skipping the vertex "0"
    # plt.plot(pts[1:, 0], pts[1:, 1], 'o')
    
    # Show the plot
    plt.show()

    # now interpolation
    new_data = np.copy(cropped_data)

    # for interpolation, grid of all column and row positions, excluding the first and last rows/cols
    cols, rows = np.meshgrid(
        np.arange(1, cropped_data.shape[2] - 1),
        np.arange(1, cropped_data.shape[1] - 1)

    )

    # flatten the grid to get a list of all (col, row) locations
    locs = np.column_stack((cols.ravel(), rows.ravel()))

    # laplace interpolation
    interpolated_values = dt.interpolate({"method": "Laplace"}, locs)

    # reshape interpolated grid back to og
    interpolated_grid = np.reshape(interpolated_values, (cropped_data.shape[1] - 2, cropped_data.shape[2] - 2))

    # fill new_data with interpolated values
    new_data[0, 1:-1, 1:-1] = interpolated_grid
    new_data = np.where(np.isnan(new_data), nodata_value, new_data)
    
    new_transform = transform * Affine.translation(1, 1)

    return new_data[0, 1:-1, 1:-1], new_transform

In [ ]:
dtm_path = "data/DTM_ams.tif"
dsm_path = "data/DSM_ams.tif"
chm_path = "D:/Geomatics/CHM/25DN2/CHM_25DN2_20.TIF"
nodata_value = -9999

In [ ]:
raster_paths = [dtm_path, chm_path, dsm_path]
overlapping_bbox = get_bbox(raster_paths)
print("Processing tile testing")

# Cropping rasters to bbox
dtm_cropped, dtm_transform, dtm_crs, dtm_mask = crop_raster(dtm_path, overlapping_bbox, no_data=nodata_value)
dsm_cropped, dsm_transform, dsm_crs, dsm_mask = crop_raster(dsm_path, overlapping_bbox, no_data=nodata_value)
chm_cropped, chm_transform, _, _ = crop_raster(chm_path, overlapping_bbox, no_data=nodata_value)

In [ ]:
XYZ_FILLED = extract_center_cells(dtm_cropped, no_data=-9999)

In [ ]:
XYZ_FILLED[0]

In [ ]:
dt = startinpy.DT()
dt.insert(XYZ_FILLED, "BBox")

In [ ]:
dt.write_geojson("myfile.geojson")

In [ ]:
pts = dt.points
trs = dt.triangles

# Create a new figure with a specified size (width, height) in inches
plt.figure(figsize=(20, 25))  # Adjust the size as needed

# Plot the triangles with thinner lines
plt.triplot(pts[:, 0], pts[:, 1], trs, linewidth=0.1)  # Adjust the linewidth as needed

# Plot the points, skipping the vertex "0"
# plt.plot(pts[1:, 0], pts[1:, 1], 'o')

# Show the plot
plt.show()

In [ ]:
convex = dt.convex_hull()

In [ ]:
print(convex)